Código Goppa

In [17]:

# bibliotecas
import numpy as np
import random
from sage.rings.finite_rings.finite_field_constructor import FiniteField
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.coding.goppa_code import GoppaCode
import math
from itertools import combinations, permutations
from sage.all import Integer
from numpy.linalg import matrix_rank


In [18]:
# verificar se o polinomio é irredutivel
def is_irreducible(poly, field):
    # Check if the polynomial is irreducible over the given field
    return poly.is_irreducible()

In [19]:
def verificar_input(m,t):
    if (m < 0 ) or (t < 0):
        return False
    else:
        return True

In [20]:
def generate_random_goppa_code(m, t):
    if verificar_input(m, t):
        n = 2**(m)
        F = FiniteField(2**m)
        R.<x> = PolynomialRing(F)
        max_iterations=(2**10)
        dim = 0
        iter_err = 0

        while dim != n - m * t and iter_err < max_iterations:
            iter_err += 1
            g = R.irreducible_element(t)

            if g.is_irreducible():
                L = [a for a in F.list() if g(a) != 0]
                np.random.shuffle(L)
                C = GoppaCode(g, L[:n])
                dim = C.dimension()

                if dim == n - m * t:
                    print(iter_err, "trials until generated")
                    generator_matrix = np.array( [ [  int(entry) for entry in row  ] for row in C.generator_matrix().change_ring(F)  ], dtype= int )
                    return dim, generator_matrix, np.linalg.matrix_rank(generator_matrix)

        return "Failure during the generation of Goppa code. Try it again"
    else:
        return "Invalid input"

In [21]:
m = 5
t = 4 
dim, generator_matrix, rank = generate_random_goppa_code(m, t)
print("Dimension of the code:", dim)
print("Rank of the Generator Matrix:", rank)
generator_matrix.shape

1 trials until generated
Dimension of the code: 12
Rank of the Generator Matrix: 12


(12, 32)

In [22]:
def Generate_S_invertible():
    condition = True
    dim = generator_matrix.shape[0]
    while condition:
        S = np.mod(np.random.permutation( dim * dim   ).reshape(dim  , dim ),2)
        if np.linalg.det(S)  !=0:
            condition = False
    return  S
def generate_permutation_matrix ():
     # Crie uma matriz de identidade
    identity_matrix = np.eye(generator_matrix.shape[1], dtype=int)

    # Gere uma permutação aleatória das linhas
    permutation = np.random.permutation(generator_matrix.shape[1])

    # Crie a matriz de permutação
    permutation_matrix = identity_matrix[permutation]
    return  permutation_matrix
def generate_random_message_vector():
    return [np.random.choice([0, 1]) for _ in range(  generator_matrix.shape[0 ]   )]
def generate_random_error_vector():
    import random
    control = True
    while control:
        binary_numbers_of_t_error = [ random.randint(0, 1) for _ in range(t)    ]
        if hamming_weight( binary_numbers_of_t_error ) == 0:
            control == False
        else:
            control =False

    dimension_vetor = [0]*generator_matrix.shape[1]
    for value in binary_numbers_of_t_error:
        random_position = np.random.choice( range(len(dimension_vetor)))
        dimension_vetor[random_position ] = value

    return dimension_vetor
def hamming_weight(binary_sequence):
    return sum(1 for bit in binary_sequence if bit == 1)
def binary_xor(vector1, vector2):
    result = []
    if len(vector1) != len(vector2):
        raise ValueError("Os vetores devem ter o mesmo comprimento")
    for i in range(   len(vector1) ):
        if  vector1[i] == vector2[i] :
            result.append( 0 )
        else:
            result.append( 1 )

    return result

#G = SGP
Public_key = np.matmul((np.matmul(Generate_S_invertible() ,generator_matrix ) )  , generate_permutation_matrix ()  ) % 2
print(  np.linalg.matrix_rank( generate_permutation_matrix () ) , np.linalg.matrix_rank( generator_matrix)  )

32 12


In [23]:
def verificar_inversa_public_key(indices_aleatorios):
    # Converte a matriz para o tipo de dados 'int'
    matrix_I = np.array( Public_key )[:, indices_aleatorios].astype(int)
    if np.linalg.det( matrix_I ) != 0:
        return True
    else:
        return False

In [24]:
random_error_vector    =   generate_random_error_vector()
weight                 =   hamming_weight( random_error_vector  )
message                =   generate_random_message_vector()
message_G              =   np.dot(message, Public_key  ) % 2
ciphertext             =   binary_xor( message_G ,   random_error_vector ) 


In [25]:
def GISD(Public_key, ciphertext, weight, j ):
    control = True
    # gerar um novo vector de erro aleatorio( este erro é diferente do erro que foi usado para cifrar a mensagem)
    random_error_vector =  generate_random_error_vector()

    while control :
        I =  np.random.choice(Public_key.shape[1], Public_key.shape[0], replace = False ) 
        error_vector_I = np.array([  random_error_vector[i] for i in  I ])
        ciphertext_I = np.array([  ciphertext[i] for i in  I ])
        # verificar se Public_key_I é invertivel
        if verificar_inversa_public_key(I) == True:
          
            # se for invertivel
            Q1 = np.array( Public_key )[:, I]
            Q2 = np.dot(Q1, Public_key ) % 2
            z = binary_xor( ciphertext, np.dot(ciphertext_I,Q2) % 2 )
          
            for value in range(j + 1):
                if hamming_weight( error_vector_I  ) == value:
                    if  hamming_weight(   binary_xor(z, np.dot(error_vector_I, Q2 )%2 ) ) == weight:
                        control = False
                        message_recovery = (np.dot((binary_xor( ciphertext_I,error_vector_I )), Q1)% 2)
                        comparacao = [message[i] == message_recovery [i] for i in range(len( message_recovery  ))]
                        #print(  message_recovery , "\t", message )
                        if all(comparacao):
                            print("message_recovery:",message_recovery , "\t original message:", message)
                            return True
                        else:
                            return 0 # will repeat again
        else:
            pass        

In [26]:

def run():
    # j <= weight
    condition = True
    while condition:
        verificou = GISD(Public_key, ciphertext,weight, j = weight  ) 
        if verificou == True:
            condition = False
            print( "Success Attack "  )
        else:
            condition = True
run()


message_recovery: [0 1 0 0 1 0 0 1 0 1 0 1] 	 original message: [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
Success Attack 
